## Deliverable 2. Create a Customer Travel Destinations Map.

In [43]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [44]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,81.28,80,100,6.91,overcast clouds
1,1,Nipawin,CA,53.3668,-104.0009,30.24,59,0,4.61,clear sky
2,2,Tondano,ID,1.3038,124.9112,78.89,70,47,0.67,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,75.70,75,97,16.93,overcast clouds
4,4,San Patricio,US,28.0170,-97.5169,74.89,90,100,11.12,overcast clouds


In [45]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [46]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,81.28,80,100,6.91,overcast clouds
2,2,Tondano,ID,1.3038,124.9112,78.89,70,47,0.67,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,75.70,75,97,16.93,overcast clouds
6,6,Key West,US,24.5557,-81.7826,82.40,65,40,17.27,scattered clouds
14,14,Baglung,NP,28.2719,83.5898,83.28,19,0,3.47,clear sky
16,16,Kilindoni,TZ,-7.9139,39.6668,79.88,83,69,10.40,moderate rain
18,18,Mersing,MY,2.4312,103.8405,85.91,84,40,0.00,scattered clouds
19,19,Georgetown,MY,5.4112,100.3354,87.73,78,40,8.05,scattered clouds
22,22,Avarua,CK,-21.2078,-159.7750,82.45,78,75,8.05,broken clouds
26,26,Mahebourg,MU,-20.4081,57.7000,75.49,78,40,13.80,scattered clouds


In [47]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             141
City                   141
Country                141
Lat                    141
Lng                    141
Max Temp               141
Humidity               141
Cloudiness             141
Wind Speed             141
Current Description    141
dtype: int64

In [48]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# All rows have values
# preferred_cities_df.dropna()

In [49]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, Current Description, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
0,Hilo,US,overcast clouds,81.28,19.7297,-155.0900,
2,Tondano,ID,scattered clouds,78.89,1.3038,124.9112,
3,Rikitea,PF,overcast clouds,75.70,-23.1203,-134.9692,
6,Key West,US,scattered clouds,82.40,24.5557,-81.7826,
14,Baglung,NP,clear sky,83.28,28.2719,83.5898,
16,Kilindoni,TZ,moderate rain,79.88,-7.9139,39.6668,
18,Mersing,MY,scattered clouds,85.91,2.4312,103.8405,
19,Georgetown,MY,scattered clouds,87.73,5.4112,100.3354,
22,Avarua,CK,broken clouds,82.45,-21.2078,-159.7750,
26,Mahebourg,MU,scattered clouds,75.49,-20.4081,57.7000,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.


# 6b. Iterate through the hotel DataFrame.
hotels = []

for i in range(len(hotel_df)):
    # 6c. Get latitude and longitude from DataFrame
    lat=hotel_df.iloc[i]['Lat']
    lng=hotel_df.iloc[i]['Lng']

    params = {
    "location":f'{lat},{lng}',
    "radius":5000,
    "types":"lodging",
    "key":g_key
}
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response =  requests.get(url,params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotels.append(response['results'][0]['name'])
    except (IndexError):
        hotels.append(None)
        print("Hotel not found... skipping.")
hotel_df["Hotel Name"]=hotels
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [51]:
# 7. Drop the rows where there is no Hotel Name.
hotel_list_df =hotel_df.dropna()
hotel_list_df.count()

City                   131
Country                131
Current Description    131
Max Temp               131
Lat                    131
Lng                    131
Hotel Name             131
dtype: int64

In [52]:
# 8a. Create the output File (CSV)
import os  
os.makedirs('Vacation_Search', exist_ok=True)  
# 8b. Export the City_Data into a csv
hotel_list_df.to_csv('Vacation_Search/WeatherPy_vacation.csv')

In [53]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_list_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_list_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))